In [1]:
import os
import argparse
import datetime
import numpy as np
import torch
from torch import nn, optim
from torch.autograd import Variable
from model import single_GAN as Model_F
from model.version_2 import model_P as Model_P
from model.version_2.train_model_P import train_model_P
from utils.data_loader import data_loader, make_pair_data
import pdb
from utils.read_bin_cfp import read_bin_cfp, load_feat

In [2]:
import easydict


In [3]:
args = easydict.EasyDict({
    "lr": 0.0002,
    "beta1": 0.5,
    "beta2": 0.999,
    "epochs": 100,
    "batch_size": 64,
    "save_dir": 'snapshot',
    "save_freq": 1,
    "load_model": False,
    "split_id": 10,
    "cuda": True,
    "data_place": "./dataset",
    "model_type": "Model_P",
})

In [4]:
if args.model_type == 'Model_F':
    args.save_dir = os.path.join(args.save_dir, 'Model_F',datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
elif args.model_type == 'Model_P':
    args.save_dir = os.path.join(args.save_dir, 'Model_P',datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
os.makedirs(args.save_dir)

print("Parameters:")
for attr, value in sorted(args.__dict__.items()):
    text ="\t{}={}\n".format(attr.upper(), value)
    print(text)
    with open('{}/Parameters.txt'.format(args.save_dir),'a') as f:
        f.write(text)

if args.model_type == 'Model_F':
    #train model model_F
    print("Do nothing")
else:
    #Train model_P
    #load front feature from model_F
    front_feat_file= './dataset/cfp-dataset/front_feat.bin'
    profile_feat_file = './dataset/cfp-dataset/profile_feat.bin'
    protocol_dir = './dataset/cfp-dataset/Protocol/Split'
    pair_type = 'FP'
    split = args.split_id

    final_front_pair, final_front_id = load_feat(front_feat_file)

    #load image
    images_f, id_labels_f, Nd, channel_num = data_loader(args.data_place, 'Front')
    images_p, id_labels_p, Nd, channel_num = data_loader(args.data_place, 'Profile')

    #make pair data
    final_images_p, final_images_f, id_labels, final_front_feat = make_pair_data(images_p, images_f, id_labels_p, final_front_pair, protocol_dir, pair_type, split)

Parameters:
	BATCH_SIZE=64

	BETA1=0.5

	BETA2=0.999

	CUDA=True

	DATA_PLACE=./dataset

	EPOCHS=100

	LOAD_MODEL=False

	LR=0.0002

	MODEL_TYPE=Model_P

	SAVE_DIR=snapshot\Model_P\2018-05-23_12-05-20

	SAVE_FREQ=1

	SPLIT_ID=10



  0%|                                                                                          | 0/500 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
 45%|███████████████████████████████████▋                                            | 223/500 [00:09<00:12, 22.34it/s]C:\ProgramData\Anaconda3\lib\site-packages\tqdm\_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:10<00:00, 47.93it/s]


In [5]:
print(np.shape(final_images_p))
print(np.shape(final_images_f))
print(np.shape(id_labels))
print(np.shape(final_front_feat))

(3150, 3, 110, 110)
(3150, 3, 110, 110)
(3150, 350)
(3150, 350, 320)


In [6]:
print(np.shape(id_labels_p))

(2000,)


In [7]:
len(id_labels_p)

2000

In [12]:
id_labels[2]

array([ 0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,
        2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,
        4,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  7,  7,
        7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,
        9,  9, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 12,
       12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14,
       14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16,
       17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19,
       19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21,
       21, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23, 23, 24, 24,
       24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26,
       26, 26, 27, 27, 27, 27, 27, 27, 27, 28, 28, 28, 28, 28, 28, 28, 29,
       29, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 30, 30, 31, 31, 31, 31,
       31, 31, 31, 32, 32

In [10]:
np.reshape(id_labels, 3150)

ValueError: cannot reshape array of size 1102500 into shape (3150,)